# Special Mention

I would like to thank the following people/software platoform/websites:
- Kaggle for providing the places for like-minded people to gather and kaggle user @Knightbearr for providing the datasets<a href="https://www.kaggle.com/datasets/knightbearr/sales-product-data/" target="_blank"><sup>1</sup></a> for free for community to use
- Stackoverflow and data science stack exchange for solving coding issues/bugs/ understand concepts
- youtuber @datatutorials1 for the inspiration of tableau ideas
- Tableau for allowing community to publish their findings for free

# Motivation

I am curious about how sales works, what does people usually buys and how does it recommend customer items to buy. Which is why I decided do this project to get some insights.

# Table of Content

1. Introduction 
2. Problem Statement
3. Import libraries
4. Functions
5. Import Data, Data Cleaning and EDA
6. Customer purchased analysis
7. Recommender System - Apriori Algorithm
8. Limitation and Further Studies
9. Conclusion

# 1. Introduction

The dataset is taken from @Knightbearr,a kaggle user, who provided me with these datasets. There are a total of 12 datasets where it is separated by month. Before analyisng, microsoft excel is used to perform ETL (Extract Transform Load), data cleaning and combining the datset into one and then after python is used to do EDA and recommender system. Recommender system is built to improve sales via upselling and cross-selling and increase of conversion rate by up to 45%<a href="https://blog.saleslayer.com/recommendation-systems-ecommerce#collab" target="_blank"><sup>2</sup></a>.

For the recommender system, 

There are also a few task to finding and the below are the task:
- How much was earned that Year?
- What was the best month for sales? How much was earned that month?
- What City had the highest number of sales?
- What time should we display adverstisement to maximize likelihood of customer's buying product?
- What products are most often sold together?
- What product sold the most? Why do you think it sold the most? 
- How much probability for next people will ordered USB-C Charging Cable?
- How much probability for next people will ordered iPhone?
- How much probability for next people will ordered Google Phone?
- How much probability other peoples will ordered Wired Headphones?

# 2. Problem Statement

- To answer the tasks above
- Find insights about the customer behaviour
- Build a recommender system to generate more sales by cross-selling and up-selling



# 3. Import Libraries

In [29]:
#Import Libraries
import os # for directory
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from itertools import combinations
from collections import Counter


#Data Mining / Association rule learning (Unsupervised Machine Learning)
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

#Remove warnings
import warnings
warnings.filterwarnings("ignore")

# 4. Functions

In [30]:
#

In [31]:
#For cleaning data for apriori
def clean_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

# 5. Import Data, Data Cleaning and EDA

In [32]:
#Before importing datasets ,check current directory
#cwd=os.getcwd() 
#display(cwd)

In [33]:
#Import datasets after ETL and cleaning using excel
df=pd.read_csv("..\\00-Datasets\\01-combined-dataset.csv")
df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141235,lightning charging cable,1,14.95,01/28/19 14:15,"185 maple st, portland, or 97035"
1,141276,lightning charging cable,1,14.95,01/21/19 22:23,"63 cherry st, los angeles, ca 90001"
2,141278,lightning charging cable,1,14.95,01/26/19 12:14,"100 cherry st, new york city, ny 10001"
3,141279,lightning charging cable,1,14.95,01/03/19 19:10,"938 14th st, boston, ma 02215"
4,141281,lightning charging cable,1,14.95,01/05/19 16:51,"274 2nd st, atlanta, ga 30301"
...,...,...,...,...,...,...
185681,318883,aaa batteries (4-pack),4,2.99,12/14/19 07:40,"93 jackson st, los angeles, ca 90001"
185682,319117,aaa batteries (4-pack),4,2.99,12/05/19 20:30,"322 hill st, los angeles, ca 90001"
185683,319376,aaa batteries (4-pack),4,2.99,12/06/19 21:39,"890 church st, dallas, tx 75001"
185684,319382,aaa batteries (4-pack),4,2.99,12/30/19 06:51,"406 walnut st, los angeles, ca 90001"


In [34]:
#Double check cleaning before EDA
display(df.duplicated().sum())

display(df.shape)

display(df.info())

display(df.describe())


0

(185686, 6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185686 entries, 0 to 185685
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Order ID          185686 non-null  int64  
 1   Product           185686 non-null  object 
 2   Quantity Ordered  185686 non-null  int64  
 3   Price Each        185686 non-null  float64
 4   Order Date        185686 non-null  object 
 5   Purchase Address  185686 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 8.5+ MB


None

,Order ID,Quantity Ordered,Price Each
count,185686.000000,185686.000000,185686.000000
mean,230411.376227,1.124544,184.519255
std,51511.717183,0.443069,332.843838
min,141234.000000,1.000000,2.990000
25%,185833.250000,1.000000,11.950000
50%,230354.500000,1.000000,14.950000
75%,275028.750000,1.000000,150.000000
max,319670.000000,9.000000,1700.000000


In [35]:
#Create sales columns for more data analysis
df["Sales"]=df["Quantity Ordered"] * df["Price Each"]
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Sales
0,141235,lightning charging cable,1,14.95,01/28/19 14:15,"185 maple st, portland, or 97035",14.95
1,141276,lightning charging cable,1,14.95,01/21/19 22:23,"63 cherry st, los angeles, ca 90001",14.95
2,141278,lightning charging cable,1,14.95,01/26/19 12:14,"100 cherry st, new york city, ny 10001",14.95
3,141279,lightning charging cable,1,14.95,01/03/19 19:10,"938 14th st, boston, ma 02215",14.95
4,141281,lightning charging cable,1,14.95,01/05/19 16:51,"274 2nd st, atlanta, ga 30301",14.95


In [36]:
#Create City column for more data analysis
df["City"]=df["Purchase Address"].map(lambda x : x.split(',')[1])
df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Sales,City
0,141235,lightning charging cable,1,14.95,01/28/19 14:15,"185 maple st, portland, or 97035",14.95,portland
1,141276,lightning charging cable,1,14.95,01/21/19 22:23,"63 cherry st, los angeles, ca 90001",14.95,los angeles
2,141278,lightning charging cable,1,14.95,01/26/19 12:14,"100 cherry st, new york city, ny 10001",14.95,new york city
3,141279,lightning charging cable,1,14.95,01/03/19 19:10,"938 14th st, boston, ma 02215",14.95,boston
4,141281,lightning charging cable,1,14.95,01/05/19 16:51,"274 2nd st, atlanta, ga 30301",14.95,atlanta
...,...,...,...,...,...,...,...,...
185681,318883,aaa batteries (4-pack),4,2.99,12/14/19 07:40,"93 jackson st, los angeles, ca 90001",11.96,los angeles
185682,319117,aaa batteries (4-pack),4,2.99,12/05/19 20:30,"322 hill st, los angeles, ca 90001",11.96,los angeles
185683,319376,aaa batteries (4-pack),4,2.99,12/06/19 21:39,"890 church st, dallas, tx 75001",11.96,dallas
185684,319382,aaa batteries (4-pack),4,2.99,12/30/19 06:51,"406 walnut st, los angeles, ca 90001",11.96,los angeles


In [37]:
#Convert to time format
df["Order Date"]=pd.to_datetime(df["Order Date"],errors="coerce")
df.info()
df.head(5)
df["Order Date"].dt.year.unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185686 entries, 0 to 185685
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Order ID          185686 non-null  int64         
 1   Product           185686 non-null  object        
 2   Quantity Ordered  185686 non-null  int64         
 3   Price Each        185686 non-null  float64       
 4   Order Date        185686 non-null  datetime64[ns]
 5   Purchase Address  185686 non-null  object        
 6   Sales             185686 non-null  float64       
 7   City              185686 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 11.3+ MB


array([2019, 2020], dtype=int64)

In [38]:
#After checking, I decided to remove year 2020 as it is incomplete.
df=df[df["Order Date"] < pd.Timestamp(2020,1,1)]
#Check results
display(df.sort_values("Order Date",ascending=True).tail(5))

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Sales,City
181050,298406,aaa batteries (4-pack),2,2.99,2019-12-31 23:42:00,"30 elm st, san francisco, ca 94016",5.98,san francisco
86357,297481,aa batteries (4-pack),1,3.84,2019-12-31 23:42:00,"82 hill st, dallas, tx 75001",3.84,dallas
181535,317048,aaa batteries (4-pack),2,2.99,2019-12-31 23:45:00,"743 adams st, san francisco, ca 94016",5.98,san francisco
101142,309773,aaa batteries (4-pack),1,2.99,2019-12-31 23:52:00,"60 hickory st, los angeles, ca 90001",2.99,los angeles
100146,297674,aaa batteries (4-pack),1,2.99,2019-12-31 23:53:00,"425 lake st, portland, or 97035",2.99,portland


In [39]:
#Trend on year
df["Year"]=df['Order Date'].dt.year

year_sales=df.groupby("Year").sum().reset_index()
year_chart=px.bar(
    year_sales,
    x="Year",
    y="Sales",
    title="Sales by year"
)
year_chart.show()

How much was earned that Year?

34.46 million was earned in that year.

In [40]:
#Month bar chart
df["Month"]=df["Order Date"].dt.month
monthly_sales=df.groupby("Month").sum()
monthly_sales

month_chart=px.bar(
    monthly_sales,
    x=monthly_sales.index,
    y="Sales",
    color="Quantity Ordered",
    title="Sales by month")

month_chart.update_layout(barmode="stack")
month_chart.show()

The best month for sale is December and 4.61 million was earned in that month.

Other note is that as sales increases, the quantity ordered also increased. This also means that company will have to prepare and pile up their inventory before these period starts. Therefore, the inventory and human resouce department will have to prepare the manpower and inventory before hand.

In [41]:
#Daily bar chart
df["Daily"]=df["Order Date"].dt.day
daily_sales=df.groupby("Daily").sum()

daily_chart=px.bar(
    daily_sales,
    x=daily_sales.index,
    y="Sales",
    color="Quantity Ordered",
    title="Sales by Day"
    )

daily_chart.update_layout(barmode="stack")
daily_chart.show()

In [42]:
#Hourly bar chart
df["Hourly"]=df["Order Date"].dt.hour
hour_sale=df.groupby("Hourly").sum()

hour_chart=px.bar(
    hour_sale,
    x=hour_sale.index,
    y="Sales",
    color="Quantity Ordered",
    title="Sales by Hour"
    )

hour_chart.update_layout(barmode="stack")
hour_chart.show()

We should display advertisement at around 1900 hours to maximise the likelihood of customers buying the product as that hour has the highest number of sales.

In [43]:
#Trend on month
month_sales=df.groupby("Month").sum()
month_trend=px.line(
    month_sales,
    x=month_sales.index,
    y="Sales",
    title="Monthly Sales Trend"
    )
month_trend.show()

In [44]:
#Trend on Daily
daily_trend=px.line(
    daily_sales,
    x=daily_sales.index,
    y="Sales",
    title ="Daily Sales Trend"
    )
daily_trend.show()

In [45]:
#Trend by hour
hour_sale=df.groupby("Hourly").sum()
hour_trend=px.line(
    hour_sale,
    x=hour_sale.index,
    y="Sales",
    title="Hourly Sales Trend"
    )

hour_trend.show()

In [46]:
#Sales by city
city_sale=df.groupby("City").sum()
city_chart=px.bar(
    city_sale,
    x=city_sale.index,
    y="Sales",
    title="City Sales"
)
city_chart.show()

San Francisco has the highest number of sales, around 8.25 million for the year ended 2019.

Other note based on this data is that it can also be used for inventory and manpower allocation.

In [47]:
#Sales by product
product_sale=df.groupby("Product").sum()
product_chart=px.bar(
    product_sale,
    x=product_sale.index,
    y="Sales",
    title="Product Sales"
)
product_chart.show()

Macbook pro made the most sales.

In [48]:
#Sales by product per city
city_product=df.groupby(["City","Product","Quantity Ordered"])["Sales"].sum().reset_index()

city_product_sold=px.bar(
    city_product,
    x="City",
    y="Sales",
    color="Product",
    title="Product Sold by City"
)
city_product_sold.show()


These chart shows the number of sales made per city and categorize by per product. It provides a guide for how much inventory and manpower to allocate for the products.

In [49]:
#product sales by month 
product_month=df.groupby(["Product","Month"])["Sales","Quantity Ordered"].sum().reset_index()

product_month_bar=px.bar(
    product_month,
    x="Month",
    y="Sales",
    color="Product",
    title="Product Sold by month"
)
product_month_bar.show()

In [50]:
#Quantity Ordered products by month

product_quantity=px.bar(
    product_month,
    x="Month",
    y="Quantity Ordered",
    color="Product",
    title="Quantity Ordered by month"
)
product_quantity.show()

In [51]:
#Products Sold
product_sold=px.bar(
    product_sale,
    x=product_sale.index,
    y="Quantity Ordered",
    color="Sales",
    title="Product Sold"
)
product_sold.show()

The product that sold the most is "aaa batteries(4-pack)". The reason being is that people are using products that requires aaa batteries. Even though it is the most sold item, it made one of the lowest sales. This means that people uses these platform to buy daily electronic necessary more often.

# 6. Customer purchased analysis

In [52]:
often_sold=df[df["Order ID"].duplicated(keep=False)]
display(often_sold)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Sales,City,Year,Month,Daily,Hourly
49,141645,lightning charging cable,1,14.95,2019-01-30 15:30:00,"98 forest st, new york city, ny 10001",14.95,new york city,2019,1,30,15
57,141738,lightning charging cable,1,14.95,2019-01-14 20:53:00,"183 cherry st, atlanta, ga 30301",14.95,atlanta,2019,1,14,20
82,141939,lightning charging cable,1,14.95,2019-01-11 14:36:00,"119 walnut st, dallas, tx 75001",14.95,dallas,2019,1,11,14
84,141947,lightning charging cable,1,14.95,2019-01-17 11:05:00,"770 5th st, san francisco, ca 94016",14.95,san francisco,2019,1,17,11
89,141970,lightning charging cable,1,14.95,2019-01-05 13:06:00,"136 11th st, boston, ma 02215",14.95,boston,2019,1,5,13
...,...,...,...,...,...,...,...,...,...,...,...,...
185617,296444,aaa batteries (4-pack),4,2.99,2019-12-17 14:24:00,"504 hickory st, atlanta, ga 30301",11.96,atlanta,2019,12,17,14
185618,296668,aaa batteries (4-pack),4,2.99,2019-12-20 16:07:00,"535 7th st, san francisco, ca 94016",11.96,san francisco,2019,12,20,16
185628,299434,aaa batteries (4-pack),4,2.99,2019-12-20 18:43:00,"664 ridge st, boston, ma 02215",11.96,boston,2019,12,20,18
185640,302888,aaa batteries (4-pack),4,2.99,2019-12-03 17:38:00,"954 north st, boston, ma 02215",11.96,boston,2019,12,3,17


In [53]:
#Find out the bundle based on Order ID
often_sold["Product bundle"]=often_sold.groupby("Order ID")["Product"].transform(lambda x : " , ".join(x))
often_sold

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Sales,City,Year,Month,Daily,Hourly,Product bundle
49,141645,lightning charging cable,1,14.95,2019-01-30 15:30:00,"98 forest st, new york city, ny 10001",14.95,new york city,2019,1,30,15,"lightning charging cable , wired headphones"
57,141738,lightning charging cable,1,14.95,2019-01-14 20:53:00,"183 cherry st, atlanta, ga 30301",14.95,atlanta,2019,1,14,20,"lightning charging cable , iphone"
82,141939,lightning charging cable,1,14.95,2019-01-11 14:36:00,"119 walnut st, dallas, tx 75001",14.95,dallas,2019,1,11,14,"lightning charging cable , 34in ultrawide monitor"
84,141947,lightning charging cable,1,14.95,2019-01-17 11:05:00,"770 5th st, san francisco, ca 94016",14.95,san francisco,2019,1,17,11,"lightning charging cable , bose soundsport hea..."
89,141970,lightning charging cable,1,14.95,2019-01-05 13:06:00,"136 11th st, boston, ma 02215",14.95,boston,2019,1,5,13,"lightning charging cable , apple airpods headp..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
185617,296444,aaa batteries (4-pack),4,2.99,2019-12-17 14:24:00,"504 hickory st, atlanta, ga 30301",11.96,atlanta,2019,12,17,14,"usb-c charging cable , aaa batteries (4-pack)"
185618,296668,aaa batteries (4-pack),4,2.99,2019-12-20 16:07:00,"535 7th st, san francisco, ca 94016",11.96,san francisco,2019,12,20,16,"aaa batteries (4-pack) , aaa batteries (4-pack)"
185628,299434,aaa batteries (4-pack),4,2.99,2019-12-20 18:43:00,"664 ridge st, boston, ma 02215",11.96,boston,2019,12,20,18,"27in fhd monitor , aaa batteries (4-pack)"
185640,302888,aaa batteries (4-pack),4,2.99,2019-12-03 17:38:00,"954 north st, boston, ma 02215",11.96,boston,2019,12,3,17,"27in fhd monitor , aaa batteries (4-pack)"


In [54]:
#Remove duplicates
often_sold_bundle=often_sold[["Order ID","Product bundle"]].drop_duplicates(keep="first")
often_sold_bundle.shape

(6876, 2)

In [55]:
#create a new variable for Counter
count = Counter()

#for loop for 2 bundle
for row in often_sold_bundle['Product bundle']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list, 2)))

#another for loop    
for key, value in count.most_common(10):
    print(key, value)

('lightning charging cable ', ' iphone') 931
('usb-c charging cable ', ' google phone') 917
('wired headphones ', ' iphone') 367
('usb-c charging cable ', ' vareebadd phone') 338
('wired headphones ', ' google phone') 299
('apple airpods headphones ', ' iphone') 299
('bose soundsport headphones ', ' google phone') 170
('wired headphones ', ' vareebadd phone') 108
('usb-c charging cable ', ' wired headphones ') 107
('lightning charging cable ', ' aa batteries (4-pack)') 98


In [56]:
#create a new variable for Counter
count = Counter()

#for loop for 3 bundle
for row in often_sold_bundle['Product bundle']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list, 3)))

#another for loop    
for key, value in count.most_common(10):
    print(key, value)

('usb-c charging cable ', ' wired headphones ', ' google phone') 78
('lightning charging cable ', ' wired headphones ', ' iphone') 47
('lightning charging cable ', ' apple airpods headphones ', ' iphone') 40
('usb-c charging cable ', ' bose soundsport headphones ', ' google phone') 32
('usb-c charging cable ', ' wired headphones ', ' vareebadd phone') 29
('wired headphones ', ' apple airpods headphones ', ' iphone') 22
('wired headphones ', ' bose soundsport headphones ', ' google phone') 18
('usb-c charging cable ', ' bose soundsport headphones ', ' vareebadd phone') 14
('lightning charging cable ', ' iphone ', ' wired headphones') 7
('wired headphones ', ' iphone ', ' lightning charging cable') 6


In [57]:
# create a new variable for Counter
count = Counter()

#for loop for 4 bundles
for row in often_sold_bundle['Product bundle']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list, 4)))

#another for loop    
for key, value in count.most_common(10):
    print(key, value)

('usb-c charging cable ', ' wired headphones ', ' bose soundsport headphones ', ' google phone') 2
('usb-c charging cable ', ' wired headphones ', ' bose soundsport headphones ', ' vareebadd phone') 2
('lightning charging cable ', ' wired headphones ', ' apple airpods headphones ', ' iphone ') 1
('lightning charging cable ', ' wired headphones ', ' apple airpods headphones ', ' google phone') 1
('lightning charging cable ', ' wired headphones ', ' iphone ', ' google phone') 1
('lightning charging cable ', ' apple airpods headphones ', ' iphone ', ' google phone') 1
(' wired headphones ', ' apple airpods headphones ', ' iphone ', ' google phone') 1
('lightning charging cable ', ' usb-c charging cable ', ' wired headphones ', ' iphone') 1
('lightning charging cable ', ' wired headphones ', ' apple airpods headphones ', ' iphone') 1
('lightning charging cable ', ' wired headphones ', ' iphone ', ' aa batteries (4-pack)') 1


The most often sold together products are :

For 2 bundle : lightening charging cable and iphone

For 3 bundle : usb-c charging cable , wired headphones , google phone

For 4 bundle : usb-c charging cable , wired headphones , bose soundsport headphones , google phone

In [58]:
#total number of products sold
total_sold=len(df)

In [59]:
#How much probability for next people will ordered USB-C Charging Cable?

#Count cable
cable_count=0

#for loop
for cable_name in df["Product"]:
    if cable_name == "usb-c charging cable":
        cable_count+=1
display(f"The total amount of google phone sold is {cable_count}.")
prob_cable=round((cable_count/total_sold),2)
display(f'The probability of next people ordering USB-C Charging Cable is {prob_cable}')

'The total amount of google phone sold is 21855.'

'The probability of next people ordering USB-C Charging Cable is 0.12'

In [60]:
#How much probability for next people will ordered iPhone?

#count iphone
iphone_count=0

#for loop
for iphone_name in df["Product"]:
    if iphone_name == "iphone":
        iphone_count+=1
display(f"The total amount of google phone sold is {iphone_count}.")
prob_iphone=round((iphone_count/total_sold),2)
display(f'The probability of next people ordering Iphone is {prob_iphone}.')

'The total amount of google phone sold is 6838.'

'The probability of next people ordering Iphone is 0.04.'

In [61]:
#How much probability for next people will ordered Google Phone?

#count google phone
g_phone_count=0

#for loop
for gphone in df["Product"]:
    if gphone=="google phone":
        g_phone_count+=1
display(f"The total amount of google phone sold is {g_phone_count}.")
prob_gphone=round((g_phone_count/total_sold),2)
display(f"The probability of next people ordering Google Phone is {prob_gphone}.")

'The total amount of google phone sold is 5521.'

'The probability of next people ordering Google Phone is 0.03.'

In [62]:
#How much probability other peoples will ordered Wired Headphones?

#count wired headphone
whphone_count=0

#for loop
for whphone in df["Product"]:
    if whphone == "wired headphones":
        whphone_count+=1
display(f"The total amount of Wired Headphone sold is  {whphone_count}.")
prob_whphone=round((whphone_count/total_sold),2)
display(f"The probability of next people ordering Wired Headphone is {prob_whphone}.")

'The total amount of Wired Headphone sold is  18848.'

'The probability of next people ordering Wired Headphone is 0.1.'

# 7. Recommender System - Apriori Algorithm

In [63]:
#this need more practice
#https://pbpython.com/market-basket-analysis.html, https://www.kaggle.com/code/datatheque/association-rules-mining-market-basket-analysis/notebook
basket = (df.groupby(['Order ID', 'Product'])['Quantity Ordered']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Order ID'))
basket

Product,20in monitor,27in 4k gaming monitor,27in fhd monitor,34in ultrawide monitor,aa batteries (4-pack),aaa batteries (4-pack),apple airpods headphones,bose soundsport headphones,flatscreen tv,google phone,iphone,lg dryer,lg washing machine,lightning charging cable,macbook pro laptop,thinkpad laptop,usb-c charging cable,vareebadd phone,wired headphones
Order ID,,,,,,,,,,,,,,,,,,,
141234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
141235,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
141236,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
141237,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
141238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
319667,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
319668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [64]:
basket_sets = basket.applymap(clean_units)
basket_sets.head(3)

Product,20in monitor,27in 4k gaming monitor,27in fhd monitor,34in ultrawide monitor,aa batteries (4-pack),aaa batteries (4-pack),apple airpods headphones,bose soundsport headphones,flatscreen tv,google phone,iphone,lg dryer,lg washing machine,lightning charging cable,macbook pro laptop,thinkpad laptop,usb-c charging cable,vareebadd phone,wired headphones
Order ID,,,,,,,,,,,,,,,,,,,
141234,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
141235,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
141236,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [81]:
#Support is relative frequency that the rules show up. High support=useful relationship, (sometimes) low support=hidden relationship.
frequent_itemsets =apriori(basket_sets, min_support=0.000001,use_colnames=True)
display(frequent_itemsets.sort_values("support",ascending=False).head(20))

,support,itemsets
16,0.122479,(usb-c charging cable)
13,0.121067,(lightning charging cable)
5,0.115411,(aaa batteries (4-pack))
4,0.115142,(aa batteries (4-pack))
18,0.105635,(wired headphones)
6,0.086998,(apple airpods headphones)
7,0.074521,(bose soundsport headphones)
2,0.042011,(27in fhd monitor)
10,0.038328,(iphone)
1,0.034887,(27in 4k gaming monitor)


People usually only do 1 purchase items and it is less common for people to buy multiple items.

In [101]:
frequent_itemsets[frequent_itemsets["itemsets"]=={"usb-c charging cable"}]

,support,itemsets
16,0.122479,(usb-c charging cable)


In [96]:
#confidence measure of the reliability of the rule. The higer the more confident it is. It is by percentage.
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.001)
display(rules.sort_values("confidence",ascending=False))

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
559,"(wired headphones, google phone, apple airpods...",(iphone),0.000006,0.038328,0.000006,1.000000,26.090377,0.000005,inf
549,"(wired headphones, lightning charging cable, u...",(iphone),0.000006,0.038328,0.000006,1.000000,26.090377,0.000005,inf
449,"(iphone, aa batteries (4-pack), wired headphones)",(lightning charging cable),0.000006,0.121067,0.000006,1.000000,8.259919,0.000005,inf
555,"(iphone, wired headphones, google phone, light...",(apple airpods headphones),0.000006,0.086998,0.000006,1.000000,11.494491,0.000005,inf
556,"(iphone, google phone, apple airpods headphone...",(wired headphones),0.000006,0.105635,0.000006,1.000000,9.466518,0.000005,inf
...,...,...,...,...,...,...,...,...,...
368,"(google phone, usb-c charging cable)",(iphone),0.005588,0.038328,0.000006,0.001003,0.026169,-0.000209,0.962637
437,"(google phone, usb-c charging cable)","(34in ultrawide monitor, bose soundsport headp...",0.005588,0.000095,0.000006,0.001003,10.526049,0.000005,1.000909
381,"(google phone, usb-c charging cable)",(vareebadd phone),0.005588,0.011569,0.000006,0.001003,0.086697,-0.000059,0.989423
428,"(google phone, usb-c charging cable)","(wired headphones, 27in fhd monitor)",0.005588,0.000123,0.000006,0.001003,8.133765,0.000005,1.000881


In [95]:
#lift is ratio of the observed support to that expected if the two rules were independent (similar to heatmap), the higher the better the relationship. if "1" it means independent
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.001)
display(rules.sort_values("lift",ascending=False))

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
859,"(iphone, google phone)","(wired headphones, apple airpods headphones, l...",0.000050,0.000022,0.000006,0.111111,4955.722222,0.000006,1.124975
858,"(wired headphones, apple airpods headphones, l...","(iphone, google phone)",0.000022,0.000050,0.000006,0.250000,4955.722222,0.000006,1.333266
851,"(iphone, google phone, apple airpods headphones)","(wired headphones, lightning charging cable)",0.000006,0.000723,0.000006,1.000000,1382.992248,0.000006,inf
866,"(wired headphones, lightning charging cable)","(iphone, google phone, apple airpods headphones)",0.000723,0.000006,0.000006,0.007752,1382.992248,0.000006,1.007807
849,"(iphone, google phone, lightning charging cable)","(wired headphones, apple airpods headphones)",0.000011,0.000561,0.000006,0.500000,892.030000,0.000006,1.998879
...,...,...,...,...,...,...,...,...,...
430,(aaa batteries (4-pack)),"(iphone, lightning charging cable)",0.115411,0.005661,0.000006,0.000049,0.008579,-0.000648,0.994387
571,"(google phone, usb-c charging cable)",(lightning charging cable),0.005588,0.121067,0.000006,0.001003,0.008285,-0.000671,0.879816
574,(lightning charging cable),"(google phone, usb-c charging cable)",0.121067,0.005588,0.000006,0.000046,0.008285,-0.000671,0.994458
80,(iphone),(27in fhd monitor),0.038328,0.042011,0.000011,0.000292,0.006962,-0.001599,0.958269


In [71]:
req=rules[(rules['lift'] >= 0.001) & (rules['confidence']>= 0.02) & (rules["support"]>=0.002)].sort_values("antecedent support",ascending=False)
req.sort_values("confidence",ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
254,(google phone),(usb-c charging cable),0.030946,0.122479,0.005588,0.180583,1.474401,0.001798,1.070909
313,(vareebadd phone),(usb-c charging cable),0.011569,0.122479,0.002063,0.178295,1.455715,0.000646,1.067926
262,(iphone),(lightning charging cable),0.038328,0.121067,0.005661,0.147704,1.220023,0.001021,1.031254
259,(google phone),(wired headphones),0.030946,0.105635,0.002365,0.076435,0.723577,-0.000904,0.968383
272,(iphone),(wired headphones),0.038328,0.105635,0.002590,0.067564,0.639592,-0.001459,0.959169
186,(iphone),(apple airpods headphones),0.038328,0.086998,0.002085,0.054402,0.625322,-0.001249,0.965528
263,(lightning charging cable),(iphone),0.121067,0.038328,0.005661,0.046761,1.220023,0.001021,1.008847
255,(usb-c charging cable),(google phone),0.122479,0.030946,0.005588,0.045627,1.474401,0.001798,1.015383
273,(wired headphones),(iphone),0.105635,0.038328,0.002590,0.024514,0.639592,-0.001459,0.985839
187,(apple airpods headphones),(iphone),0.086998,0.038328,0.002085,0.023968,0.625322,-0.001249,0.985287


Interesting Insights:
- people who buy google or vareebadd phone first will have a higher chance of buying either usb-c charging cable or wired headphone. Possible to consider upsell vareebadd to google phone and cross sell usb-c cable and wireheadphone.
- people who buy iphone first will have a higher chance of buying lightning charging cable or apple airpods headphones or wired headphones and vice versa. Possible to consider upsell or cross sell these products.
- people who buy wired headphone have a higher chances of buying either iphone or google phone, it is possible that they already have a phone and it is possible to up sell phone products.

In [104]:
#Save csv files
df.to_csv("..\\00-Datasets\\02-To_tableau.csv")

## Data Dictionary


# 8. Limitation and Further Studies

The limitation for the project are the following :
1. time constraint to complete the project,
2. the limitations of Apriori Algorithm which is it is [performance inefficient](https://en.wikipedia.org/wiki/Apriori_algorithm). 
3. More revelent datas for analysis. For example, datas for multiple years, education and income of the customers, and other factors that may affect the buying decisions of the customers.

For further studies, it is possible to explore other algorithm such as [max miner](https://www2.cs.sfu.ca/CourseCentral/741/jpei/readings/baya98.pdf) or [fast miner](https://www.researchgate.net/publication/253105580_Efficiently_mining_maximal_frequent_patterns_fast-miner) to solve the performance inefficiency, getting more data to visualise the trends.


# 9. Conclusion

The key takeaways for this project :
1. Sales
- 34.46 million was earned in Year 2019.
- best month for sale is December and 4.61 million was earned.
- San Francisco has the highest number of sales, around 8.25 million for the year ended 2019.
- most sales product is Macbook pro.
- most sold products is aaa batteries(4-pack). Even though it is the most sold item, it made one of the lowest sales. This means that people buy more lower cost proudcts as compared expensive products.

2. Advertisement recommendation
- recommended advertisement display is around 1900 hours to maximise the likelihood of customers buying the product as that hour has the highest number of sales.

3. Customer purchased analysis
- most often 2 bundle sold products are lightening charging cable and iphone.
- most often 3 bundle sold products are usb-c charging cable , wired headphones , google phone.
- most often 4 bundle sold products are usb-c charging cable , wired headphones , bose soundsport headphones , google phone.
- probability of next people ordering USB-C Charging Cable is 0.12.
- probability of next people ordering Iphone is 0.04.
- probability of next people ordering Google Phone is 0.03.
- probability of next people ordering Wired Headphone is 0.1.
- customers usually only do 1 purchase item.

4. Up-sell recommendations
- usb-c charging cable or wired headphone customers to google phone
- lighwired headphonetning charging cable or apple airpods headphones or wired headphones customers to iphone
- wired headphone customers to iphone or google phone

5. Cross-sell recommendations
- google or vareebadd phone customers to usb-c charging cable or wired headphone customers
- iphone customers to lightning charging cable or apple airpods headphones or wired headphones
